In [1]:
# Disable TensorFlow debugging info and warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 2: Info and warnings not displayed 

In [2]:
# Add massimal tools folder to path
import sys
sys.path.append("/massimal/python/tools")

In [3]:
import tensorflow as tf
import numpy as np
import skimage.io
import sklearn.decomposition
import matplotlib.pyplot as plt
import pathlib
#import tqdm
import pickle
import hyspec_cnn
import datetime

In [4]:
# Paths
base_dir = pathlib.Path('/massimal/data/Vega_Sola/Hyperspectral/20220823/Area')
train_tiles_path = base_dir / '3a_PCA_TrainValidationSplit/Training/PCA-Tiles/20220823_Vega_Sola_Train_Tiles'
val_tiles_path = base_dir / '3a_PCA_TrainValidationSplit/Validation/PCA-Tiles/20220823_Vega_Sola_Val_Tiles'

unet_model_save_dir = base_dir / 'M_UnetModels'
unet_model_save_dir.mkdir(exist_ok=True)
tensorboard_log_dir = base_dir / 'M_TensorBoardLogs'
tensorboard_log_dir.mkdir(exist_ok=True)

In [5]:
# Check if GPU is used
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
# Parameters
OUTPUT_CHANNELS = 8
BATCH_SIZE = 8
DEPTH = 4

In [7]:
# Load datasets 
train_dataset = tf.data.Dataset.load(str(train_tiles_path))
val_dataset = tf.data.Dataset.load(str(val_tiles_path))

In [8]:
# Get number of tiles in each dataset, and dataset shape
n_tiles_train = train_dataset.cardinality()
n_tiles_val = val_dataset.cardinality()
tile_nrows,tile_ncols,tile_nchannels = train_dataset.element_spec[0].shape.as_list()
print(f'Number of training tiles: {n_tiles_train}')
print(f'Number of validation tiles: {n_tiles_val}')
print(f'Tile data shape (PCA tiles): {(tile_nrows,tile_ncols,tile_nchannels)}')

Number of training tiles: 3266
Number of validation tiles: 711
Tile data shape (PCA tiles): (128, 128, 8)


In [9]:
# From https://www.tensorflow.org/tutorials/images/segmentation#optional_imbalanced_classes_and_class_weights
def add_sample_weights(image, label, name):
    class_weights = tf.constant([0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]) # Hard-coded for 7 classes
    class_weights = class_weights/tf.reduce_sum(class_weights)

    # Create an image of `sample_weights` by using the label at each pixel as an 
    # index into the `class weights` .
    sample_weights = tf.gather(class_weights, indices=tf.cast(label, tf.int32))

    return image, label, sample_weights

In [10]:
# Shuffle training dataset (tiles are originally ordered by image) and add sample weights
train_dataset = train_dataset.shuffle(buffer_size=n_tiles_train)
train_dataset = train_dataset.map(add_sample_weights)
val_dataset = val_dataset.map(add_sample_weights)

In [11]:
# Batch datasets
train_dataset_batch = train_dataset.batch(BATCH_SIZE)
val_dataset_batch = val_dataset.batch(BATCH_SIZE)

In [12]:
# Create the U-Net model
unet = hyspec_cnn.unet(input_channels=tile_nchannels,
                       output_channels=OUTPUT_CHANNELS,
                       first_layer_channels=32,
                       depth = DEPTH,
               )
unet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, None, None,  0           []                               
                                 8)]                                                              
                                                                                                  
 augmentation (Sequential)      (None, None, None,   0           ['input_image[0][0]']            
                                8)                                                                
                                                                                                  
 initial_convolution (Conv2D)   (None, None, None,   2336        ['augmentation[0][0]']           
                                32)                                                           

In [13]:
# Print layers with sublayers
for layer in unet.layers:
    print('----')
    print(layer.name)
    if hasattr(layer,'layers'):
        for l in layer.layers:
            print('\t'+l.name)
       # print(layer.layers)

----
input_image
----
augmentation
	random_flip
----
initial_convolution
----
downsamp_res_1/2
	conv2d
	batch_normalization
	leaky_re_lu
----
downsamp_res_1/4
	conv2d_1
	batch_normalization_1
	leaky_re_lu_1
----
downsamp_res_1/8
	conv2d_2
	batch_normalization_2
	leaky_re_lu_2
----
downsamp_res_1/16
	conv2d_3
	batch_normalization_3
	leaky_re_lu_3
----
upsamp_res_1/8
	conv2d_transpose
	batch_normalization_4
	re_lu
----
skipconnection_res_1/8
----
upsamp_res_1/4
	conv2d_transpose_1
	batch_normalization_5
	re_lu_1
----
skipconnection_res_1/4
----
upsamp_res_1/2
	conv2d_transpose_2
	batch_normalization_6
	re_lu_2
----
skipconnection_res_1/2
----
upsamp_res_1/1
	conv2d_transpose_3
	batch_normalization_7
	re_lu_3
----
skipconnection_res_1/1
----
classification


In [14]:
# Define callbacks
model_save_filename = str(unet_model_save_dir) + '/unet_model.depth' + str(DEPTH) +'.epoch{epoch:02d}-loss{val_loss:.6f}-acc{val_sparse_categorical_accuracy:.3f}.hdf5'
callbacks =[tf.keras.callbacks.ModelCheckpoint(filepath = model_save_filename,
                                               save_best_only=True,
                                               verbose = 1),
            tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, verbose=1),
            tf.keras.callbacks.TensorBoard(log_dir= tensorboard_log_dir)]

In [15]:
model_save_filename

'/massimal/data/Vega_Sola/Hyperspectral/20220823/Area/M_UnetModels/unet_model.depth4.epoch{epoch:02d}-loss{val_loss:.6f}-acc{val_sparse_categorical_accuracy:.3f}.hdf5'

In [16]:
# Compile model
unet.compile(optimizer=tf.keras.optimizers.RMSprop(0.0001), 
             loss="sparse_categorical_crossentropy",
             weighted_metrics=['sparse_categorical_accuracy'], # Need weights to ignore background
             metrics = []) # Sparse because classes are numbered, not one-hot

In [17]:
# print(tensorboard_log_dir)
#%tensorboard --logdir /massimal/data/Vega_Sola/Hyperspectral/20220823/Area/M_TensorBoardLogs

In [18]:
# Fit model to dataset
history = unet.fit(train_dataset.batch(BATCH_SIZE),
                   epochs=100,
                   validation_data=val_dataset.batch(BATCH_SIZE),
                   callbacks=callbacks)

Epoch 1/100
409/409 [==============================] - ETA: 0s - loss: 0.1154 - sparse_categorical_accuracy: 0.4706
Epoch 1: val_loss improved from inf to 0.08209, saving model to /massimal/data/Vega_Sola/Hyperspectral/20220823/Area/M_UnetModels/unet_model.depth4.epoch01-loss0.082087-acc0.587.hdf5
409/409 [==============================] - 112s 255ms/step - loss: 0.1154 - sparse_categorical_accuracy: 0.4706 - val_loss: 0.0821 - val_sparse_categorical_accuracy: 0.5866 - lr: 1.0000e-04
Epoch 2/100
409/409 [==============================] - ETA: 0s - loss: 0.0918 - sparse_categorical_accuracy: 0.5695
Epoch 2: val_loss improved from 0.08209 to 0.07243, saving model to /massimal/data/Vega_Sola/Hyperspectral/20220823/Area/M_UnetModels/unet_model.depth4.epoch02-loss0.072435-acc0.661.hdf5
409/409 [==============================] - 105s 253ms/step - loss: 0.0918 - sparse_categorical_accuracy: 0.5695 - val_loss: 0.0724 - val_sparse_categorical_accuracy: 0.6609 - lr: 1.0000e-04
Epoch 3/100
409/40

Epoch 22/100
409/409 [==============================] - ETA: 0s - loss: 0.0433 - sparse_categorical_accuracy: 0.7973
Epoch 22: val_loss did not improve from 0.05416
409/409 [==============================] - 104s 246ms/step - loss: 0.0433 - sparse_categorical_accuracy: 0.7973 - val_loss: 0.0664 - val_sparse_categorical_accuracy: 0.6828 - lr: 1.0000e-04
Epoch 23/100
409/409 [==============================] - ETA: 0s - loss: 0.0423 - sparse_categorical_accuracy: 0.8008
Epoch 23: val_loss improved from 0.05416 to 0.04984, saving model to /massimal/data/Vega_Sola/Hyperspectral/20220823/Area/M_UnetModels/unet_model.depth4.epoch23-loss0.049841-acc0.764.hdf5
409/409 [==============================] - 103s 249ms/step - loss: 0.0423 - sparse_categorical_accuracy: 0.8008 - val_loss: 0.0498 - val_sparse_categorical_accuracy: 0.7643 - lr: 1.0000e-04
Epoch 24/100
409/409 [==============================] - ETA: 0s - loss: 0.0414 - sparse_categorical_accuracy: 0.8076
Epoch 24: val_loss did not improv

Epoch 44/100
409/409 [==============================] - ETA: 0s - loss: 0.0231 - sparse_categorical_accuracy: 0.8928
Epoch 44: val_loss did not improve from 0.04980
409/409 [==============================] - 102s 247ms/step - loss: 0.0231 - sparse_categorical_accuracy: 0.8928 - val_loss: 0.0550 - val_sparse_categorical_accuracy: 0.7542 - lr: 4.0000e-06
Epoch 45/100
409/409 [==============================] - ETA: 0s - loss: 0.0225 - sparse_categorical_accuracy: 0.8974
Epoch 45: val_loss did not improve from 0.04980
409/409 [==============================] - 102s 247ms/step - loss: 0.0225 - sparse_categorical_accuracy: 0.8974 - val_loss: 0.0535 - val_sparse_categorical_accuracy: 0.7597 - lr: 4.0000e-06
Epoch 46/100
409/409 [==============================] - ETA: 0s - loss: 0.0220 - sparse_categorical_accuracy: 0.8999
Epoch 46: val_loss did not improve from 0.04980
409/409 [==============================] - 103s 248ms/step - loss: 0.0220 - sparse_categorical_accuracy: 0.8999 - val_loss: 0

Epoch 67/100
409/409 [==============================] - ETA: 0s - loss: 0.0214 - sparse_categorical_accuracy: 0.9022
Epoch 67: val_loss did not improve from 0.04980
409/409 [==============================] - 102s 247ms/step - loss: 0.0214 - sparse_categorical_accuracy: 0.9022 - val_loss: 0.0553 - val_sparse_categorical_accuracy: 0.7589 - lr: 1.6000e-07
Epoch 68/100
409/409 [==============================] - ETA: 0s - loss: 0.0216 - sparse_categorical_accuracy: 0.9003
Epoch 68: val_loss did not improve from 0.04980
409/409 [==============================] - 103s 248ms/step - loss: 0.0216 - sparse_categorical_accuracy: 0.9003 - val_loss: 0.0556 - val_sparse_categorical_accuracy: 0.7573 - lr: 1.6000e-07
Epoch 69/100
409/409 [==============================] - ETA: 0s - loss: 0.0211 - sparse_categorical_accuracy: 0.9038
Epoch 69: val_loss did not improve from 0.04980
409/409 [==============================] - 102s 247ms/step - loss: 0.0211 - sparse_categorical_accuracy: 0.9038 - val_loss: 0

KeyboardInterrupt: 